### Implement 3DES

In [1]:
from Crypto.Cipher import DES3
from Crypto.Random import get_random_bytes
import base64

# Pad text so length is a multiple of 8 (DES block size)
def pad(text):
    while len(text) % 8 != 0:
        text += ' '
    return text

# Generating a valid 3DES key (16 or 24 bytes)
def generate_key():
    while True:
        key = get_random_bytes(24)  # 24 bytes for 3DES
        try:
            DES3.adjust_key_parity(key)
            return key
        except ValueError:
            continue

# Encrypt function
def encrypt(key, plaintext):
    cipher = DES3.new(key, DES3.MODE_ECB)
    padded_text = pad(plaintext)
    encrypted = cipher.encrypt(padded_text.encode('utf-8'))
    return base64.b64encode(encrypted).decode('utf-8')

# Decrypt function
def decrypt(key, ciphertext):
    cipher = DES3.new(key, DES3.MODE_ECB)
    decrypted = cipher.decrypt(base64.b64decode(ciphertext)).decode('utf-8')
    return decrypted.strip()

if __name__ == "__main__":
    key = generate_key()
    print("3DES Key (Base64):", base64.b64encode(key).decode('utf-8'))

    text = "Hello123ABC"
    print("Plaintext:", text)

    encrypted = encrypt(key, text)
    print("Encrypted:", encrypted)

    decrypted = decrypt(key, encrypted)
    print("Decrypted:", decrypted)


3DES Key (Base64): KDoZWltfDkDKriA8Pf1pHNYOibtLG+cd
Plaintext: Hello123ABC
Encrypted: vgDhRcZHjmmwuTLjB4DDkA==
Decrypted: Hello123ABC


### Implement AES

In [2]:
# AES-128 Implementation (Raw, Encryption + Decryption)

# AES S-box
s_box = [
    [0x63,0x7c,0x77,0x7b,0xf2,0x6b,0x6f,0xc5,0x30,0x01,0x67,0x2b,0xfe,0xd7,0xab,0x76],
    [0xca,0x82,0xc9,0x7d,0xfa,0x59,0x47,0xf0,0xad,0xd4,0xa2,0xaf,0x9c,0xa4,0x72,0xc0],
    [0xb7,0xfd,0x93,0x26,0x36,0x3f,0xf7,0xcc,0x34,0xa5,0xe5,0xf1,0x71,0xd8,0x31,0x15],
    [0x04,0xc7,0x23,0xc3,0x18,0x96,0x05,0x9a,0x07,0x12,0x80,0xe2,0xeb,0x27,0xb2,0x75],
    [0x09,0x83,0x2c,0x1a,0x1b,0x6e,0x5a,0xa0,0x52,0x3b,0xd6,0xb3,0x29,0xe3,0x2f,0x84],
    [0x53,0xd1,0x00,0xed,0x20,0xfc,0xb1,0x5b,0x6a,0xcb,0xbe,0x39,0x4a,0x4c,0x58,0xcf],
    [0xd0,0xef,0xaa,0xfb,0x43,0x4d,0x33,0x85,0x45,0xf9,0x02,0x7f,0x50,0x3c,0x9f,0xa8],
    [0x51,0xa3,0x40,0x8f,0x92,0x9d,0x38,0xf5,0xbc,0xb6,0xda,0x21,0x10,0xff,0xf3,0xd2],
    [0xcd,0x0c,0x13,0xec,0x5f,0x97,0x44,0x17,0xc4,0xa7,0x7e,0x3d,0x64,0x5d,0x19,0x73],
    [0x60,0x81,0x4f,0xdc,0x22,0x2a,0x90,0x88,0x46,0xee,0xb8,0x14,0xde,0x5e,0x0b,0xdb],
    [0xe0,0x32,0x3a,0x0a,0x49,0x06,0x24,0x5c,0xc2,0xd3,0xac,0x62,0x91,0x95,0xe4,0x79],
    [0xe7,0xc8,0x37,0x6d,0x8d,0xd5,0x4e,0xa9,0x6c,0x56,0xf4,0xea,0x65,0x7a,0xae,0x08],
    [0xba,0x78,0x25,0x2e,0x1c,0xa6,0xb4,0xc6,0xe8,0xdd,0x74,0x1f,0x4b,0xbd,0x8b,0x8a],
    [0x70,0x3e,0xb5,0x66,0x48,0x03,0xf6,0x0e,0x61,0x35,0x57,0xb9,0x86,0xc1,0x1d,0x9e],
    [0xe1,0xf8,0x98,0x11,0x69,0xd9,0x8e,0x94,0x9b,0x1e,0x87,0xe9,0xce,0x55,0x28,0xdf],
    [0x8c,0xa1,0x89,0x0d,0xbf,0xe6,0x42,0x68,0x41,0x99,0x2d,0x0f,0xb0,0x54,0xbb,0x16]
]

# Inverse S-box
inv_s_box = [[0]*16 for _ in range(16)]
for i in range(16):
    for j in range(16):
        val = s_box[i][j]
        inv_s_box[val >> 4][val & 0x0F] = (i << 4) | j

# Round constant
Rcon = [0x01,0x02,0x04,0x08,0x10,0x20,0x40,0x80,0x1B,0x36]

# Helpers
def sub_bytes(state):
    for i in range(4):
        for j in range(4):
            row = state[i][j] >> 4
            col = state[i][j] & 0x0F
            state[i][j] = s_box[row][col]
    return state

def inv_sub_bytes(state):
    for i in range(4):
        for j in range(4):
            row = state[i][j] >> 4
            col = state[i][j] & 0x0F
            state[i][j] = inv_s_box[row][col]
    return state

def shift_rows(state):
    state[1] = state[1][1:] + state[1][:1]
    state[2] = state[2][2:] + state[2][:2]
    state[3] = state[3][3:] + state[3][:3]
    return state

def inv_shift_rows(state):
    state[1] = state[1][-1:] + state[1][:-1]
    state[2] = state[2][-2:] + state[2][:-2]
    state[3] = state[3][-3:] + state[3][:-3]
    return state

def xtime(a):
    return (((a << 1) ^ 0x1B) & 0xFF) if (a & 0x80) else (a << 1)

def mix_single_column(a):
    t = a[0] ^ a[1] ^ a[2] ^ a[3]
    u = a[0]
    a[0] ^= t ^ xtime(a[0] ^ a[1])
    a[1] ^= t ^ xtime(a[1] ^ a[2])
    a[2] ^= t ^ xtime(a[2] ^ a[3])
    a[3] ^= t ^ xtime(a[3] ^ u)
    return a

def mix_columns(state):
    for i in range(4):
        col = [state[j][i] for j in range(4)]
        col = mix_single_column(col)
        for j in range(4):
            state[j][i] = col[j]
    return state

# GF(2^8) multiplication
def gmul(a, b):
    p = 0
    for _ in range(8):
        if b & 1:
            p ^= a
        hi = a & 0x80
        a = (a << 1) & 0xFF
        if hi:
            a ^= 0x1B
        b >>= 1
    return p

def inv_mix_columns(state):
    for i in range(4):
        a = [state[j][i] for j in range(4)]
        state[0][i] = gmul(a[0],14)^gmul(a[1],11)^gmul(a[2],13)^gmul(a[3],9)
        state[1][i] = gmul(a[0],9)^gmul(a[1],14)^gmul(a[2],11)^gmul(a[3],13)
        state[2][i] = gmul(a[0],13)^gmul(a[1],9)^gmul(a[2],14)^gmul(a[3],11)
        state[3][i] = gmul(a[0],11)^gmul(a[1],13)^gmul(a[2],9)^gmul(a[3],14)
    return state

def add_round_key(state, key_schedule, rnd):
    for i in range(4):
        for j in range(4):
            state[j][i] ^= key_schedule[rnd*4+i][j]
    return state

def key_expansion(key):
    key_columns = [[key[4*i+j] for j in range(4)] for i in range(4)]
    i = 4
    while len(key_columns) < 44:
        word = list(key_columns[-1])
        if i % 4 == 0:
            word = word[1:] + word[:1]  # RotWord
            word = [s_box[b >> 4][b & 0x0F] for b in word]  # SubWord
            word[0] ^= Rcon[i//4 - 1]
        word = [a ^ b for a, b in zip(word, key_columns[-4])]
        key_columns.append(word)
        i += 1
    return key_columns

def bytes2matrix(text):
    return [list(text[i:i+4]) for i in range(0, len(text), 4)]

def matrix2bytes(matrix):
    return bytes(sum(matrix, []))

# AES Encrypt
def aes_encrypt(plaintext, key):
    state = bytes2matrix(plaintext)
    key_schedule = key_expansion(key)
    state = add_round_key(state, key_schedule, 0)

    for rnd in range(1, 10):
        state = sub_bytes(state)
        state = shift_rows(state)
        state = mix_columns(state)
        state = add_round_key(state, key_schedule, rnd)

    state = sub_bytes(state)
    state = shift_rows(state)
    state = add_round_key(state, key_schedule, 10)

    return matrix2bytes(state)

# AES Decrypt
def aes_decrypt(ciphertext, key):
    state = bytes2matrix(ciphertext)
    key_schedule = key_expansion(key)

    state = add_round_key(state, key_schedule, 10)
    state = inv_shift_rows(state)
    state = inv_sub_bytes(state)

    for rnd in range(9, 0, -1):
        state = add_round_key(state, key_schedule, rnd)
        state = inv_mix_columns(state)
        state = inv_shift_rows(state)
        state = inv_sub_bytes(state)

    state = add_round_key(state, key_schedule, 0)
    return matrix2bytes(state)

if __name__ == "__main__":
    key = b"Thats my Kung Fu"      # 16-byte key
    plaintext = b"Two One Nine Two" # 16-byte block

    ciphertext = aes_encrypt(plaintext, key)
    decrypted = aes_decrypt(ciphertext, key)

    print("Plaintext: ", plaintext)
    print("Key:       ", key)
    print("Ciphertext:", ciphertext.hex())
    print("Decrypted: ", decrypted)

Plaintext:  b'Two One Nine Two'
Key:        b'Thats my Kung Fu'
Ciphertext: 9a98c8091a238804f3eea1605c1cc8b2
Decrypted:  b'Two One Nine Two'
